In [ ]:
import os
import yaml
import pandas as pd

# config.yamlを読み込む
with open("config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

# secondary_foldersのパスを取得
secondary_folders = config["secondary_folders"]

# secondary_folders 直下にあるディレクトリを収集
patient_folders = []
for folder in secondary_folders:
    if os.path.exists(folder) and os.path.isdir(folder):
        subdirs = [
            os.path.join(folder, subdir)
            for subdir in os.listdir(folder)
            if os.path.isdir(os.path.join(folder, subdir))
        ]
        patient_folders.extend(subdirs)

# データフレームに格納
df = pd.DataFrame(patient_folders, columns=["patient_folder_path"])

# データフレームの表示
print(df)

In [ ]:
# .CR2 ファイルを.png ファイルに変換する。

import rawpy
import cv2
import numpy as np

# 患者フォルダの取得
patient_folders = []
for folder in secondary_folders:
    if os.path.exists(folder) and os.path.isdir(folder):
        subdirs = [
            os.path.join(folder, subdir)
            for subdir in os.listdir(folder)
            if os.path.isdir(os.path.join(folder, subdir))
        ]
        patient_folders.extend(subdirs)

# .cr2ファイルを抽出
cr2_files = []
for patient_folder in patient_folders:
    for root, _, files in os.walk(patient_folder):
        for file in files:
            if file.lower().endswith('.cr2'):
                cr2_files.append(os.path.join(root, file))

# .cr2ファイルを処理して.png形式で保存
for cr2_file in cr2_files:
    try:
        # rawpyで.cr2ファイルを読み込み
        with rawpy.imread(cr2_file) as raw:
            rgb = raw.postprocess()

        # 保存先のファイルパスを生成 (.cr2 -> .png)
        save_path = os.path.splitext(cr2_file)[0] + '.png'

        # OpenCVで保存
        cv2.imwrite(save_path, cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))
        print(f"Saved: {save_path}")

    except Exception as e:
        print(f"Error processing {cr2_file}: {e}")

In [ ]:
# .tgaファイルを抽出
tga_files = []
for patient_folder in patient_folders:
    for root, _, files in os.walk(patient_folder):
        for file in files:
            if file.lower().endswith('.tga'):  # 大文字小文字を無視
                tga_files.append(os.path.join(root, file))

for tga_file in tga_files:
    try:
        # OpenCVで.tgaファイルを読み込み
        img = cv2.imread(tga_file)

        # 読み込み結果を確認
        if img is None:
            print(f"Skipped (cannot read): {tga_file}")
            continue

        # 保存先のファイルパスを生成 (.tga -> .png)
        save_path = os.path.splitext(tga_file)[0] + '.png'

        # OpenCVで保存
        cv2.imwrite(save_path, img)
        print(f"Saved: {save_path}")

    except Exception as e:
        print(f"Error processing {tga_file}: {e}")

In [ ]:
# 対象となる拡張子
target_extensions = ('.jpg', '.jpeg', '.tif', '.tiff')

# エラーが発生したファイルリスト
error_files = []

# 処理するファイルを収集
image_files = []
for patient_folder in patient_folders:
    for root, _, files in os.walk(patient_folder):
        for file in files:
            if file.lower().endswith(target_extensions):  # 大文字小文字を無視
                image_files.append(os.path.join(root, file))

# 画像を変換して保存
for image_file in image_files:
    try:
        # OpenCVで画像を読み込み
        img = cv2.imread(image_file)

        # 読み込みエラーチェック
        if img is None:
            print(f"Skipped (cannot read): {image_file}")
            error_files.append(image_file)
            continue

        # 画像のサイズ取得
        height, width = img.shape[:2]

        # 長辺が3000pxを超える場合は縮小
        max_size = 3000
        if max(height, width) > max_size:
            scale = max_size / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
            print(f"Resized: {image_file} to {new_width}x{new_height}")

        # 保存先のパスを生成 (.png形式)
        base_name, _ = os.path.splitext(os.path.basename(image_file))
        save_dir = os.path.dirname(image_file)
        save_path = os.path.join(save_dir, f"{base_name}.png")

        # 同名ファイルがある場合、連番を付与
        counter = 1
        while os.path.exists(save_path):
            save_path = os.path.join(save_dir, f"{base_name}_{counter}.png")
            counter += 1

        # 画像を保存
        cv2.imwrite(save_path, img)
        print(f"Saved: {save_path}")

    except Exception as e:
        print(f"Error processing {image_file}: {e}")
        error_files.append(image_file)

# エラーのあったファイルを保存
if error_files:
    with open("error_files.txt", "w", encoding="utf-8") as f:
        for error_file in error_files:
            f.write(f"{error_file}\n")
    print("Error files saved to error_files.txt")

In [ ]:
import os
import pandas as pd
import yaml

# config.yamlの読み込み
with open("config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

# secondary_foldersの取得
secondary_folders = config["secondary_folders"]
dataset_root = config["dataset_root"]
target_extension = ".png"

# エラー記録用ログファイル
error_log_file = "image_processing_errors.log"

# ログファイルの初期化
with open(error_log_file, "w", encoding="utf-8") as log:
    log.write("Image processing errors:\n")

# 患者フォルダの確認とファイル収集
image_data = []
for folder in secondary_folders:
    if not os.path.exists(folder) or not os.path.isdir(folder):
        with open(error_log_file, "a", encoding="utf-8") as log:
            log.write(f"Folder not found or is not a directory: {folder}\n")
        print(f"Warning: Folder not found or is not a directory: {folder}")
        continue

    # 患者フォルダの収集
    subdirs = [
        os.path.join(folder, subdir)
        for subdir in os.listdir(folder)
        if os.path.isdir(os.path.join(folder, subdir))
    ]

    for subdir in subdirs:
        try:
            # 各患者フォルダ内の.pngファイルを収集
            for root, _, files in os.walk(subdir):
                for file in files:
                    if file.lower().endswith(target_extension):
                        relative_path = os.path.relpath(os.path.join(root, file), start=os.getcwd())
                        image_data.append({"dirname": os.path.basename(subdir), "path": relative_path})
        except Exception as e:
            with open(error_log_file, "a", encoding="utf-8") as log:
                log.write(f"Error processing folder {subdir}: {e}\n")
            print(f"Error processing folder {subdir}: {e}")

# データフレームの作成
df = pd.DataFrame(image_data).drop_duplicates().reset_index(drop=True)
df.index.name = "index"

# 結果を.parquetに保存
output_file = "00_all_images.parquet"
df.to_parquet(output_file, index=True)
print(f"DataFrame saved to {output_file}")

# データセットの検証
print(f"Number of images processed: {len(df)}")
print("Sample data:")
print(df.head())

# データ保存確認
try:
    test_df = pd.read_parquet(output_file)
    print(f"Parquet file read successfully. Number of rows: {len(test_df)}")
except Exception as e:
    print(f"Error reading the saved parquet file: {e}")


In [ ]:
import os
import hashlib
import cv2
import polars as pl
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # プログレスバー用

# ファイルサイズをKBに変換
def get_file_size_kb(path):
    try:
        return os.path.getsize(path) / 1024
    except Exception:
        return None

# ファイルの作成日時と更新日時を取得
def get_file_times(path):
    try:
        created_time = datetime.fromtimestamp(os.path.getctime(path))
    except Exception:
        created_time = None
    try:
        modified_time = datetime.fromtimestamp(os.path.getmtime(path))
    except Exception:
        modified_time = None
    return created_time, modified_time

# 画像の幅・高さを取得し、必要なら縮小
def get_image_dimensions(path, max_size=3000):
    try:
        img = cv2.imread(path)
        if img is None:
            return None, None, False
        height, width = img.shape[:2]
        if max(height, width) > max_size:
            scale = max_size / max(height, width)
            width, height = int(width * scale), int(height * scale)
            img = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
            cv2.imwrite(path, img)  # 縮小後の画像を保存
        return width, height, True
    except Exception:
        return None, None, False

# 画像のカラーモードを取得
def get_color_mode(path):
    try:
        img = cv2.imread(path)
        if img is None:
            return None
        channels = img.shape[2] if len(img.shape) > 2 else 1
        return 'RGB' if channels == 3 else 'RGBA' if channels == 4 else 'BW'
    except Exception:
        return None

# ファイルのハッシュ値を取得
def get_file_hash(path):
    try:
        hasher = hashlib.sha256()
        with open(path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hasher.update(chunk)
        return hasher.hexdigest()
    except Exception:
        return None

# メタデータを取得する関数
def process_image_metadata(path):
    try:
        readable = cv2.imread(path) is not None
        if not readable:
            return {"readable": False}

        file_size = get_file_size_kb(path)
        created_time, modified_time = get_file_times(path)
        width, height, resized = get_image_dimensions(path)
        color_mode = get_color_mode(path)
        file_hash = get_file_hash(path)

        return {
            "readable": True,
            "width": width,
            "height": height,
            "created_time": created_time,
            "modified_time": modified_time,
            "color_mode": color_mode,
            "file_size": file_size,
            "hash": file_hash,
        }
    except Exception as e:
        return {"readable": False, "error": str(e)}

# Parquetファイルの読み込み
df = pl.read_parquet("00_all_images.parquet")

# プログレスバー付きで並列処理
metadata = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_image_metadata, path): path for path in df["path"]}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing images"):
        metadata.append(future.result())

# メタデータをPolarsのDataFrameに変換
metadata_df = pl.DataFrame(metadata)

# 元のDataFrameにメタデータを追加
result_df = df.with_columns(metadata_df)

# 結果をParquetファイルに保存
result_df.write_parquet("01_add_metadata.parquet")

# 完了メッセージ
print("Metadata extraction completed and saved to 01_add_metadata.parquet.")


In [1]:
# 同一ハッシュ値を持つ画像の除外
# 同一患者フォルダを対象とする
# 古い作成日時のものを残す

import random
import pandas as pd

# Parquetファイルの読み込み
df = pd.read_parquet("01_add_metadata.parquet")

# 作成日時を datetime 型に変換（必要なら）
df["created_time"] = pd.to_datetime(df["created_time"], errors="coerce")

# サブグループごとの処理
df["exclude_same_folder"] = False  # デフォルトはFalse
grouped = df.groupby(["hash", "dirname"])

for (hash_value, dirname), group in grouped:
    if len(group) > 1:  # 同一ハッシュ値かつ同一フォルダ内で複数ファイルがある場合
        try:
            # 作成日時が最も古いものを特定
            oldest_time = group["created_time"].min()
            candidates = group[group["created_time"] == oldest_time]
            
            if len(candidates) > 1:  # 作成日時が一致する場合
                # ランダムに1つだけ残す
                selected_idx = random.choice(candidates.index)
            else:
                # 最古の作成日時のインデックス
                selected_idx = candidates.index[0]

            # 除外対象を設定
            df.loc[group.index.difference([selected_idx]), "exclude_same_folder"] = True

        except Exception as e:
            # エラーログの出力（必要なら）
            print(f"Error processing hash {hash_value} in folder {dirname}: {e}")

# 結果を確認
print(df[["hash", "dirname", "created_time", "exclude_same_folder"]].head())

                                                hash dirname  \
0  aa026e82dc4ff273858e008faef466d7074df8fa32750d...      61   
1  e6b3bbc1b8728b78fb12b109b50805806317b0c635d067...      61   
2  543f55671e6eee830a9f39b4199c4bb255445d88cc36dc...      61   
3  024d549a2895e4147a98d6d9ad530cf8bb9a3b1172fa34...      61   
4  e7402fded8b129aca4d8810fc01a3828dd3c5077f7cfbf...      61   

                created_time  exclude_same_folder  
0 2024-11-22 20:05:13.475698                False  
1 2024-11-22 20:05:13.295251                False  
2 2024-11-22 20:05:13.871609                False  
3 2024-11-22 20:05:13.666497                False  
4 2024-11-22 20:05:14.076683                False  


In [2]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

# フォント設定（日本語対応）
plt.rcParams["font.family"] = "Hiragino Sans"

# データフレームの準備（仮のサンプル）
df["exclude_cross_folder"] = "include"  # 初期値
# exclude_same_folder 列が True の場合、exclude_cross_folder を 'exclude' に設定

df.loc[df["exclude_same_folder"] == True, "exclude_cross_folder"] = "exclude"

# 同一ハッシュ値を持つかつ異なる患者フォルダに存在するファイルを抽出し、exclude_cross_folder を 'initial' に設定
hash_groups = df.groupby("hash")
for hash_value, group in hash_groups:
    # 異なる患者フォルダに分散している場合
    if group["dirname"].nunique() > 1:
        df.loc[group.index, "exclude_cross_folder"] = "initial"

# 処理対象の絞り込み
cross_folder_candidates = df[(df["exclude_same_folder"] == False) & (df["exclude_cross_folder"] == "initial")]
hash_groups = cross_folder_candidates.groupby("hash")

# 対象となるハッシュ値リスト
remaining_hashes = hash_groups.filter(lambda g: g["dirname"].nunique() > 1)["hash"].unique()
current_index = 0  # 処理中のハッシュインデックス

# 残りのセット数を表示
def update_progress():
    print(f"現在のセット: {current_index + 1}/{len(remaining_hashes)} (残り {len(remaining_hashes) - current_index} セット)")

# 画像表示と選択プロセス
def process_hash(hash_value):
    global df

    # 出力をクリア
    clear_output(wait=True)

    # 進捗を表示
    update_progress()

    # 対象の画像を取得
    candidates = cross_folder_candidates[cross_folder_candidates["hash"] == hash_value]
    unique_folders = candidates["dirname"].unique()

    # 描画
    fig, axes = plt.subplots(len(unique_folders), 11, figsize=(25, 5 * len(unique_folders)))
    if len(unique_folders) == 1:
        axes = [axes]  # フォルダ数が1の場合に合わせる

    for i, folder in enumerate(unique_folders):
        # 候補画像を取得
        candidate_images = candidates[candidates["dirname"] == folder]
        for j, (_, row) in enumerate(candidate_images.iterrows()):
            img = plt.imread(row["path"])
            axes[i][j].imshow(img)
            axes[i][j].set_title(f"候補: {folder}", fontsize=10)
            axes[i][j].axis("off")

        # ランダム比較画像を取得（10枚に変更）
        other_images = df[(df["dirname"] == folder) & (df["hash"] != hash_value)]
        random_images = random.sample(other_images["path"].tolist(), min(10, len(other_images)))
        for j, img_path in enumerate(random_images, start=len(candidate_images)):
            img = plt.imread(img_path)
            axes[i][j].imshow(img)
            axes[i][j].set_title(f"比較: {folder}", fontsize=8)  # 小さめのフォント
            axes[i][j].axis("off")

        # 余分なサブプロットを非表示にする
        for k in range(len(candidate_images) + len(random_images), 11):
            axes[i][k].axis("off")

    plt.tight_layout()
    plt.show()

    # ユーザー選択
    print("\n以下の選択肢から入力してください：")
    print("1. 候補画像を選択: 対象の記号 (例: a, b, ...) を入力")
    print("2. スキップ: 'skip' を入力")

    # 記号を割り当てる
    folder_map = {chr(97 + idx): folder for idx, folder in enumerate(unique_folders)}
    for key, folder in folder_map.items():
        print(f"{key}: {folder}")

    user_input = input("入力: ").strip()
    if user_input == "skip":
        print("このセットをスキップします。")
        return
    elif user_input in folder_map:
        selected_folder = folder_map[user_input]
        # 選ばれたフォルダ以外を除外
        exclude_indexes = candidates[candidates["dirname"] != selected_folder].index
        df.loc[exclude_indexes, "exclude_cross_folder"] = "exclude"
        df.loc[candidates[candidates["dirname"] == selected_folder].index, "exclude_cross_folder"] = "include"
        print(f"フォルダ '{selected_folder}' の画像を残しました。")
    else:
        print("無効な入力です。このセットをスキップします。")

# 全体処理ループ
while current_index < len(remaining_hashes):
    process_hash(remaining_hashes[current_index])
    current_index += 1

# 完了メッセージ
clear_output(wait=True)
print("すべてのセットの処理が完了しました！")

# データフレームを保存
df.to_parquet("02_processed_dataframe.parquet", index=False)
print("処理結果が 'processed_dataframe.parquet' に保存されました。")


すべてのセットの処理が完了しました！
処理結果が 'processed_dataframe.parquet' に保存されました。


In [6]:
import pandas as pd

# Parquet ファイルのロード
df = pd.read_parquet("02_processed_dataframe.parquet")

# exclude_cross_folder 列の値の件数を集計
summary = df["exclude_cross_folder"].value_counts()

# 結果を表示
print("集計結果:")
print(summary)

# 'initial' が存在するか確認
if 'initial' in summary.index:
    print("\n'exclude_cross_folder' 列に 'initial' の値があります。処理を続行します。")
    
    # 'initial' の画像を抽出
    initial_images = df[df["exclude_cross_folder"] == "initial"]
    print(f"処理対象の画像は {len(initial_images)} 件です。")
    
    # 必要なら、ここに表示用のコードを追加してください
else:
    print("\nすべての 'exclude_cross_folder' 列の値が処理済みです。")


集計結果:
exclude_cross_folder
include    15825
exclude      216
Name: count, dtype: int64

すべての 'exclude_cross_folder' 列の値が処理済みです。


In [7]:
import pandas as pd

# Parquet ファイルをロード
df = pd.read_parquet("02_processed_dataframe.parquet")

# too_small 列を作成
# 条件: width または height が 256 未満の場合 True、それ以外は False
df["too_small"] = (df["width"] < 256) | (df["height"] < 256)

# 結果を確認
print("集計結果:")
print(df["too_small"].value_counts())

# 処理済みデータフレームを保存
df.to_parquet("03_processed_dataframe_with_too_small.parquet")
print("\n結果を '03_processed_dataframe_with_too_small.parquet' に保存しました。")

集計結果:
too_small
False    15901
True       140
Name: count, dtype: int64

結果を '03_processed_dataframe_with_too_small.parquet' に保存しました。


In [8]:
import pandas as pd

# Parquet ファイルをロード
df = pd.read_parquet("03_processed_dataframe_with_too_small.parquet")

# 指定条件に合致する行を抽出
filtered_df = df[
    (df["exclude_same_folder"] == False) & 
    (df["exclude_cross_folder"] == "include") & 
    (df["too_small"] == False)
]

# フィルタ後のデータを保存
filtered_df.to_parquet("images.parquet")

# 結果を確認
print("抽出後のデータ件数:", len(filtered_df))
print("\n結果を 'images.parquet' に保存しました。")


抽出後のデータ件数: 14895

結果を 'images.parquet' に保存しました。
